In [1]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for pre-parsing

from pyparsing import Word, alphas as pp_alpha, nums as pp_nums
import pyparsing as pp
import re

In [3]:
# handshapes

handshapelist = [
  '1',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '25',
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H-U',
  'I',
  'K',
  'L',
  'M',
  'N',
  'O',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'C-L',
  'U-L',
  'B-L',
  'P-K',
  'Q-G',
  'L-X',
  'I-L-Y',
  '5-C',
  '5-C-L',
  '5-C-tt',
  'alt-M',
  'alt-N',
  'alt-P',
  'B-xd',
  'baby-O',
  'bent-1',
  'bent-B',
  'bent-B-L',
  'bent-horns',
  'bent-L',
  'bent-M',
  'bent-N',
  'bent-U',
  'cocked-8',
  'cocked-F',
  'cocked-S',
  'crvd-5',
  'crvd-B',
  'crvd-flat-B',
  'crvd-L',
  'crvd-sprd-B',
  'crvd-U',
  'crvd-V',
  'fanned-flat-O',
  'flat-B',
  'flat-C',
  'flat-F',
  'flat-G',
  'flat-O-2',
  'flat-O',
  'full-M',
  'horns',
  'loose-E',
  'O-2-horns',
  'open-8',
  'open-F',
  'sml-C-3',
  'tight-C-2',
  'tight-C',
  'X-over-thumb'
]

In [4]:
# regex rules

number_regexp = r"[0-9]+(?:\.[0-9]+)?s?"
alpha_regexp = r"(?!((?:THUMB-)?(?:IX-|POSS-|SELF-)))[A-Z0-9](?:[A-Z0-9/'-]*[A-Z0-9])?(?:\.|:[0-9])?"
lookahead_regexp = r"(?:(?![a-z])|(?=wg))"

word_all_regexp = r"""(?x)
    (?: %s | %s )
    %s
""" % (number_regexp, alpha_regexp, lookahead_regexp)

In [5]:
# conventions kept for parsing

cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"])
ns_prefix = pp.Literal("ns")
fs_prefix = pp.Literal("fs")
lex_exceptions = pp.one_of(["part", "WHAT"])
aspect_text = pp.Literal("aspect")
index_core_ix = pp.Literal("IX")
other_index_core = pp.one_of(["POSS", "SELF"])
handshape = pp.one_of(handshapelist)
person = pp.one_of(["1p", "2p", "3p"])
arc = pp.Literal("arc") 
loc = pp.Literal("loc")
pl = pp.Literal("pl")
compound = pp.Literal("+")
hashtag = pp.Literal("#")
sym = pp.Literal(">")
par1 = pp.Literal("(")
par2 = pp.Literal(")")
dash = pp.Literal("-")
contraction = pp.Literal("^")
colon = pp.Literal(":")
omit_quote = pp.Literal("xx")
period = pp.Literal(".")
alpha = Word(pp_alpha, max=1)
num = Word(pp_nums, max=1)
word = pp.Regex(word_all_regexp, flags=re.X)

In [6]:
# grammar rules

full_grammar = pp.OneOrMore(
    cl_prefix |               # classifiers like CL, DCL, etc.
    word |
    ns_prefix |               # non-specific ns
    fs_prefix |               # fingerspelling fs
    index_core_ix |           # IX
    other_index_core |        # POSS, SELF
    person |                  # 1p, 2p, 3p
    lex_exceptions |          # part, WHAT
    aspect_text |             # aspect
    arc |                     # arc
    loc |                     # loc
    pl |                      # pl
    handshape |               # handshapes like B, 1, 5, etc.
    compound |                # +
    hashtag |                 # #
    sym |                     # >
    contraction |             # ^
    colon |                   # :
    par1 | par2 |             # ( and )
    omit_quote |              # xx
    period |                  # .
    dash |
    num |                    # numbers last resort
    alpha                     # fallback LAST
)

In [7]:
# testing grammar parsing

trial = full_grammar.parse_string("SCL:1xx", parse_all=True).asList()
trial2 = full_grammar.parse_string("IX-1p BCLxx FIND/FIND-OUT fs-HER", parse_all=True).asList()

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['IX', '-', '1p', 'BCL', 'xx', 'FIND/FIND-OUT', 'fs', '-', 'HER']


In [8]:
# tokenize based on predefined grammar rules

def custom_asl_tokenize(text):
    try:
        return full_grammar.parse_string(text, parse_all=True).asList()
    except pp.ParseException as pe:
        print(f"Failed to parse: {pe}")
        return []

In [9]:
def custom_eng_tokenize(text):
    # Add spaces around punctuation (preserving it)
    text = re.sub(r'([^\w\s])', r' \1 ', text)
    # Convert to lowercase
    text = text.lower()
    # Split on whitespace
    tokens = text.split()
    return tokens

In [10]:
# testing custom_asl_tokenize

trial = custom_asl_tokenize("SCL:1xx")
trial2 = custom_asl_tokenize('SCL:1xx SHOWER WASH FEEL THUMBS-UP/GOOD')

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['SCL', ':', '1', 'xx', 'SHOWER', 'WASH', 'FEEL', 'THUMBS-UP/GOOD']


In [11]:
# model parameters / hyperparameters

BATCH_SIZE = 64
EPOCHS = 30  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 128
ENG_VOCAB_SIZE = 1692 + 4
ASL_VOCAB_SIZE = 1130 + 4
num_samples = 1400

EMBED_DIM = 128
INTERMEDIATE_DIM = 1024
NUM_HEADS = 8
data_path = "/Users/adrianajimenez/Desktop/Downloads/REUAICT/Real-Code/2025-ASL-data/data/sent_pairs.txt"

In [12]:
# generate
    # 1) list of eng-asl sentence pairs
    # 2) set of unique english vocab
    # 3) set of unique asl vocab

text_pairs = []
eng_tokens = set()
asl_tokens = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
 
for line in lines[: min(num_samples, len(lines) - 1)]:
    pair = []
    eng_text, asl_text = line.split("\t")
    pair.append(eng_text)
    pair.append(asl_text)
    text_pairs.append(pair)

for pair in text_pairs:
    sent_tokens = custom_eng_tokenize(pair[0])
    for token in sent_tokens:
        if token not in eng_tokens:
            eng_tokens.add(token)
            
for pair in text_pairs:
    sent_tokens = custom_asl_tokenize(pair[1])
    for token in sent_tokens:
        if token not in asl_tokens:
            asl_tokens.add(token)

eng_tokens = sorted(list(eng_tokens))
asl_tokens = sorted(list(asl_tokens))

print("eng_tokens:", eng_tokens)
print("output_tokens", asl_tokens)
num_encoder_tokens = len(eng_tokens)
num_decoder_tokens = len(asl_tokens)
print("num_eng_tokens", num_encoder_tokens)
print("num_asl_tokens", num_decoder_tokens)
print(asl_tokens)

Failed to parse: Expected end of text, found '/'  (at char 19), (line:1, col:20)
Failed to parse: Expected end of text, found '/'  (at char 23), (line:1, col:24)
eng_tokens: ['!', '"', '$', "'", '(', ')', '*', ',', '-', '.', '/', '1', '10', '100', '11', '110', '150', '3', '30', '300', '350', '4', '45', '5', '50', '500', '50s', '60s', '7', '70', '75', '8', '80', '85', '88', '9', '90', '96', ':', ';', '?', 'a', 'abdominal', 'able', 'about', 'accepted', 'accident', 'across', 'act', 'action', 'actually', 'admit', 'advantage', 'adventure', 'advice', 'after', 'again', 'against', 'ago', 'agree', 'ah', 'ahead', 'ahh', 'ail', 'air', 'ali', 'all', 'alone', 'along', 'already', 'alright', 'also', 'am', 'amazing', 'an', 'and', 'angeles', 'angled', 'angry', 'animal', 'ankle', 'ann', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'applaud', 'apple', 'approach', 'are', 'area', 'arm', 'arms', 'around', 'around12', 'arrest', 'arrested', 'arrive', 'arrived', 'arrives',

In [13]:
# glimpse pairs

for _ in range(5):
    print(random.choice(text_pairs))

['John will have seen Mary.', 'fs-JOHN FUTURE FINISH SEE fs-MARY']
['Especially when football is the only legal sport where you can hit someone that hard and not get fined for it.', 'SPECIAL/EXCEPT+fs-LY WHEN IX-3p FOOTBALL REALLY ONLY LEGAL/LAW SPORT IX-2p CAN BPCL:bent-Bxx SOMETHING/ONE #SO HARD AND NOT GET PRICE FOR fs-IT']
['There was waving while we were rowing, and people fell off the raft.', 'ICLxx SCL:B-Lxx SOME SCL:bent-Vxx FINISH']
["The teacher likes books, but doesn't like movies.", 'TEACHER LIKE BOOK MOVIE NOT']
['Sue is buying that blue car.', 'fs-SUE BUY IX-3p CAR BLUE']


In [14]:
# split data

random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1400 total pairs
980 training pairs
210 validation pairs
210 test pairs


In [15]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(500).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

In [16]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

asl_samples = [text_pair[1] for text_pair in train_pairs]
asl_vocab = train_word_piece(asl_samples, ASL_VOCAB_SIZE, reserved_tokens)

2025-07-15 15:06:20.580747: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 15:06:21.484469: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
print("English Tokens: ", eng_vocab[100:110])
print("ASL Tokens: ", asl_vocab[100:110])

English Tokens:  ['house', 'buy', 'teacher', 'for', 'on', 'said', '##es', 'will', 'you', 'have']
ASL Tokens:  ['##T', 'HOUSE', '##N', 'BCLxx', '##Y', 'SAME', 'arc', 'GIFT', '##ER', 'ICLxx']


In [18]:
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
asl_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(
    vocabulary=asl_vocab, lowercase=False
)

In [19]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

asl_input_ex = text_pairs[0][1]
print(f"ASL input: '{asl_input_ex}'") 
asl_tokens_ex = asl_tokenizer.tokenize(asl_input_ex)
print("ASL Gloss sentence: ", asl_input_ex)
print("Tokens: ", asl_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    asl_tokenizer.detokenize(asl_tokens_ex),
)

English sentence:  The sun sets.  Lightning strikes.  Rain falls.
Tokens:  tf.Tensor(
[ 82  68 440  99  68 335  79  11  36 196 338  68  94 164 308 363  11  41
 153 230  55 178  79  11], shape=(24,), dtype=int32)
Recovered text after detokenizing:  The sun sets . Lightning strikes . Rain falls .

ASL input: 'SUNSET LIGHTNING RAIN'
ASL Gloss sentence:  SUNSET LIGHTNING RAIN
Tokens:  tf.Tensor([ 41 362 102 324  34 359 226 172 100 268 210], shape=(11,), dtype=int32)
Recovered text after detokenizing:  SUNSET LIGHTNING RAIN


In [20]:
def preprocess_batch(eng, asl):
    batch_size = ops.shape(asl)[0]

    eng = eng_tokenizer(eng)
    asl = asl_tokenizer(asl)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `asl` and pad it as well.
    asl_start_end_packer = keras_hub.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=asl_tokenizer.token_to_id("[START]"),
        end_value=asl_tokenizer.token_to_id("[END]"),
        pad_value=asl_tokenizer.token_to_id("[PAD]"),
    )
    asl = asl_start_end_packer(asl)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": asl[:, :-1],
        },
        asl[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, asl_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    asl_texts = list(asl_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, asl_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [21]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 128)
inputs["decoder_inputs"].shape: (64, 128)
targets.shape: (64, 128)


2025-07-15 15:06:23.031659: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [22]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), name="encoder_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_hub.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(shape=(None,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_hub.layers.TokenAndPositionEmbedding(
    vocabulary_size=ASL_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_hub.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(ASL_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

In [ ]:
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, None, 128) │    233,472 │ encoder_inputs[0… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 128) │    329,856 │ token_and_positi… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, None,      │    703,982 │ decoder_inputs[0… │
│ (Functional)        │ 1134)             │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,267,310 (4.83 MB)

 Trainable params: 1,267,310 (4.83 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 411ms/step - accuracy: 0.6887 - loss: 3.1069 - val_accuracy: 0.8832 - val_loss: 0.8971
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 379ms/step - accuracy: 0.8802 - loss: 0.9302 - val_accuracy: 0.8919 - val_loss: 0.6672
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 382ms/step - accuracy: 0.8878 - loss: 0.7162 - val_accuracy: 0.8978 - val_loss: 0.5901
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 379ms/step - accuracy: 0.8919 - loss: 0.6283 - val_accuracy: 0.9004 - val_loss: 0.5474
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 379ms/step - accuracy: 0.8946 - loss: 0.5783 - val_accuracy: 0.9010 - val_loss: 0.5213
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 384ms/step - accuracy: 0.8961 - loss: 0.5486 - val_accuracy: 0.9031 - val_loss: 0.5032
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 410ms/step - accuracy: 0.8973 - loss: 0.5260 - val_accuracy: 0.9036 - val_loss: 0.4897
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 434ms/step - accuracy: 0.8975 - loss: 0.5083 - val_accuracy: 0.

In [ ]:
def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenize the encoder input.
    encoder_input_tokens = ops.convert_to_tensor(eng_tokenizer(input_sentences))
    if len(encoder_input_tokens[0]) < MAX_SEQUENCE_LENGTH:
        pads = ops.full((1, MAX_SEQUENCE_LENGTH - len(encoder_input_tokens[0])), 0)
        encoder_input_tokens = ops.concatenate(
            [encoder_input_tokens, pads], 1
        )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 40
    start = ops.full((batch_size, 1), asl_tokenizer.token_to_id("[START]"))
    pad = ops.full((batch_size, length - 1), asl_tokenizer.token_to_id("[PAD]"))
    prompt = ops.concatenate((start, pad), axis=-1)

    generated_tokens = keras_hub.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[asl_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    generated_sentences = asl_tokenizer.detokenize(generated_tokens)
    return generated_sentences


test_eng_texts = [pair[0] for pair in test_pairs]
for i in range(2):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequences([input_sentence])
    translated = decode_sequences([input_sentence])[0]
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

I0000 00:00:1752606133.576587 9373366 service.cc:152] XLA service 0x145474ae0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752606133.576616 9373366 service.cc:160]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1752606133.856157 9373366 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


** Example 0 **
Mother did not buy the car for John.
fs - JOHN READ BOOK

** Example 1 **
All of sudden, a strong storm hit.  I was a bit nervous driving through it. 
IX - 1p LOOK STRE IX - 1p 5xx



In [ ]:
rouge_1 = keras_hub.metrics.RougeN(order=1)
rouge_2 = keras_hub.metrics.RougeN(order=2)

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequences([input_sentence])
    translated_sentence = decode_sequences([input_sentence])[0]
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    rouge_1(reference_sentence, translated_sentence)
    rouge_2(reference_sentence, translated_sentence)

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())

2025-07-15 15:02:18.655586: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: Expected size[0] in [0, 50], but got 69


InvalidArgumentError: Exception encountered when calling PositionEmbedding.call().

[1m{{function_node __wrapped__Slice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected size[0] in [0, 50], but got 69 [Op:Slice][0m

Arguments received by PositionEmbedding.call():
  • inputs=tf.Tensor(shape=(1, 69, 128), dtype=float32)
  • start_index=0